<a href="https://colab.research.google.com/github/frank-morales2020/MLxDL/blob/main/AI_AOC_DEMO.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install langchain-community -q # Installs langchain-community

In [ ]:
!pip install colab-env --quiet

In [3]:
!nvidia-smi

Sat Jan 11 21:21:53 2025       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA L4                      Off | 00000000:00:03.0 Off |                    0 |
| N/A   43C    P8              12W /  72W |      1MiB / 23034MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [4]:
import warnings

warnings.filterwarnings("ignore", message="You seem to be using the pipelines sequentially on GPU")

In [ ]:
import colab_env
import os

access_token_write = os.getenv("HUGGINGFACE_ACCESS_TOKEN_WRITE")

from huggingface_hub import login

login(
  token=access_token_write,
  add_to_git_credential=True
)

In [ ]:
import os
from langchain.llms import HuggingFaceHub  # Import HuggingFaceHub


# Get your Hugging Face API token from environment variables
# OR directly assign it if not found
huggingfacehub_api_token = os.environ.get("HUGGINGFACE_ACCESS_TOKEN_WRITE")
if huggingfacehub_api_token is None:
    huggingfacehub_api_token = "YOUR_HUGGINGFACE_API_TOKEN"  # Replace with your actual token

# Initialize Mistral 7B using HuggingFaceHub, providing the API token
llm = HuggingFaceHub(
    repo_id="mistralai/Mistral-7B-Instruct-v0.1",
    model_kwargs={"temperature": 0.1, "max_new_tokens": 512},
    huggingfacehub_api_token=huggingfacehub_api_token  # Pass the token here
)

In [7]:
import os
from langchain.llms import HuggingFaceHub
import colab_env
# ... (your existing code) ...

# Get your Hugging Face API token from environment variables
huggingfacehub_api_token = os.environ.get("HUGGINGFACE_TOKEN")

#print(huggingfacehub_api_token)

# Check if the token was found, and raise an error if not
if huggingfacehub_api_token is None:
    raise ValueError(
        "Hugging Face API token not found. "
        "Please set the `HUGGINGFACEHUB_API_TOKEN` environment variable."
    )

# Initialize Mistral 7B using HuggingFaceHub, providing the API token
llm = HuggingFaceHub(
    repo_id="mistralai/Mistral-7B-Instruct-v0.1",
    model_kwargs={"temperature": 0.1, "max_new_tokens": 512},
    huggingfacehub_api_token=huggingfacehub_api_token  # Pass the token here
)


In [ ]:
!pip install faiss-gpu -q

In [ ]:
import os
import random

from transformers import AutoTokenizer, AutoModelForCausalLM
from sentence_transformers import SentenceTransformer
import faiss
import numpy as np
import torch

# --- Simulated External Data Sources ---


def get_flight_data(flight_id):
    """Simulates fetching flight data from an API."""
    # Simulate a storm affecting the flight
    storm_active = random.choice([True, False])

    flight_data = {
        "flight_id": flight_id,
        "aircraft_id": "AA123",  # Added aircraft_id to the flight data
        "origin": "Paris",
        "destination": "London",
        "status": "en-route",
        "altitude": 35000,  # feet
        "speed": 500,  # knots
        "position": {
            "latitude": 50.0,
            "longitude": 2.0
        },
        "weather": {
            "temperature": -50,  # Celsius
            "wind": {
                "speed": 20,  # knots
                "direction": "West"
            },
            "storm": {
                "active": storm_active,
                "severity": "severe" if storm_active else "none",
                "location": {
                    "latitude": 51.0,
                    "longitude": 1.0
                }
            }
        }
    }
    return flight_data


def get_maintenance_records(aircraft_id):
    """Simulates fetching maintenance records from a database."""
    maintenance_records = [{
        "date": "2024-12-15",
        "description": "Scheduled engine inspection",
        "status": "completed"
    }, {
        "date": "2025-01-05",
        "description": "Replaced hydraulic pump",
        "status": "completed"
    }]
    return maintenance_records


def suggest_rerouting(flight_data):
    """
    Simulates a tool that suggests rerouting options based on weather.
    """
    if flight_data["weather"]["storm"]["active"]:
        suggestion = "Consider rerouting flight {} to avoid the storm. Possible alternative route: Paris - Brussels - London.".format(
            flight_data["flight_id"])
    else:
        suggestion = "No rerouting suggestions at this time."
    return suggestion


# --- FAISS Setup for Semantic Search ---

# Sample documents for FAISS (replace with your actual knowledge base)
documents = [
    "Aircraft maintenance is crucial for flight safety.",
    "Severe weather can cause flight delays and disruptions.",
    "Rerouting options should consider fuel efficiency and passenger comfort.",
    "In case of emergency, pilots should follow established procedures.",
    "Communication between pilots and air traffic control is essential.",
    "Regular training ensures crew members are prepared for various situations.",
]

# Initialize a SentenceTransformer model for generating embeddings
encoder = SentenceTransformer('all-mpnet-base-v2')
embeddings = encoder.encode(documents)

# Create a FAISS index
dimension = embeddings.shape[1]
index = faiss.IndexFlatL2(dimension)
index.add(embeddings.astype('float32'))


def semantic_search(query, index, k=2):
    """Performs semantic search over the documents using FAISS."""
    query_embedding = encoder.encode([query])[0].astype('float32')
    D, I = index.search(np.array([query_embedding]), k)
    return [documents[i] for i in I[0]]


# --- Cache-Augmented Generation ---


def preload_knowledge(query, index, k=2):
    """Preloads relevant knowledge into the LLM's context window."""
    results = semantic_search(query, index, k)
    context = " ".join(results)
    return context


# --- Agent Implementation ---

# Initialize the tokenizer and model
tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-2-7b-chat-hf")  # Reverted to Llama 2 7B chat
model = AutoModelForCausalLM.from_pretrained(
    "meta-llama/Llama-2-7b-chat-hf",
    torch_dtype=torch.float16,
    device_map="auto"
)

In [28]:
# Set the pad_token_id for the model explicitly to avoid the warning
model.generation_config.pad_token_id = tokenizer.pad_token_id if tokenizer.pad_token_id is not None else tokenizer.eos_token_id

def generate_response(prompt,
                      tokenizer,
                      model,
                      max_new_tokens=1024,  # Increased for longer responses
                      temperature=0.7):
    """Generates a response from the Llama 2 model."""
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    outputs = model.generate(**inputs,
                              max_new_tokens=max_new_tokens,
                              temperature=temperature,
                              do_sample=True)
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return response

In [29]:
def run_agent(question, index):
    """Executes the AI agent."""
    # Preload knowledge using FAISS and CAG
    context = preload_knowledge(question, index)

    # Initialize the prompt with the preloaded context
    prompt = f"""You are an AI agent assisting in an airline operation control center.

    Relevant context: {context}

    Available tools:

    * get_flight_data(flight_id): Get information about a flight, including route, weather, etc.
    * suggest_rerouting(flight_id): Suggest rerouting options for a flight.
    * get_maintenance_records(aircraft_id): Get maintenance records for an aircraft.

    Instructions:

    1. **Always** start by thinking about what to do.
    2. **Clearly** state your thoughts.
    3. Choose the **best** action from the tools above.
    4. **Only** provide 'Action Input' if the tool requires it.
    5. **Always** provide an 'Observation' after each action.
    6. Once you have all the information, provide a 'Final Answer' to the original question.

    Use this format:

    Question: the input question you must answer
    Thought: ...
    Action: ...
    Action Input: ... (if needed)
    Observation: ...

    Begin!

    Question: {question}
    """

    # Generate a response from the model (no loop)
    response = generate_response(prompt, tokenizer, model)
    print(response)

    # Check if the model has a final answer
    if "Final Answer:" in response:
        final_answer = response.split("Final Answer:")[-1].strip()
        return final_answer
    else:
        # If no "Final Answer" is found, try to extract information
        try:
            action_line = next(
                (line for line in response.split("\n")
                 if line.startswith("Action:")), None)
            input_line = next(
                (line for line in response.split("\n")
                 if line.startswith("Action Input:")), None)

            if action_line and input_line:
                action = action_line.split("Action:")[-1].strip()
                action_input = input_line.split("Action Input:")[-1].strip()

                # Execute the action and add the observation to the prompt
                if action == "get_flight_data":
                    observation = get_flight_data(action_input)

                    # Extract aircraft ID from flight data
                    try:
                        aircraft_id = observation["aircraft_id"]
                    except KeyError:
                        aircraft_id = "unknown"

                elif action == "suggest_rerouting":
                    # Call suggest_rerouting with the flight data dictionary
                    flight_data = get_flight_data(action_input)
                    observation = suggest_rerouting(flight_data)

                elif action == "get_maintenance_records":
                    if action_input == "not provided":
                        # If aircraft ID is not provided, use the extracted aircraft_id
                        observation = get_maintenance_records(aircraft_id)
                    else:
                        observation = get_maintenance_records(action_input)
                else:
                    observation = "Invalid action."

                return observation  # Return the observation for now

            else:
                return "I'm sorry, I couldn't understand your request. Please rephrase."

        except IndexError:
            print("Error: Could not parse model response.")
            return "I'm sorry, I couldn't understand the model's response."

In [30]:
# Define a function to simulate user input with flight ID
def get_user_input():
    flight_id = "FA" + str(random.randint(1000, 9999))
    user_input = f"""
    Flight {flight_id} is requesting an update on their route and weather,
    as well as any relevant maintenance records for the aircraft.
    Can you provide this information and suggest any necessary actions?
    """
    return user_input

In [31]:
# Get and run user input (pass the FAISS index to run_agent)
user_query = get_user_input()
response = run_agent(user_query, index)
print(response)  # Print the final answer

You are an AI agent assisting in an airline operation control center.

    Relevant context: Severe weather can cause flight delays and disruptions. Aircraft maintenance is crucial for flight safety.

    Available tools:

    * get_flight_data(flight_id): Get information about a flight, including route, weather, etc.
    * suggest_rerouting(flight_id): Suggest rerouting options for a flight.
    * get_maintenance_records(aircraft_id): Get maintenance records for an aircraft.

    Instructions:
    
    1. **Always** start by thinking about what to do.
    2. **Clearly** state your thoughts.
    3. Choose the **best** action from the tools above.
    4. **Only** provide 'Action Input' if the tool requires it.
    5. **Always** provide an 'Observation' after each action.
    6. Once you have all the information, provide a 'Final Answer' to the original question.

    Use this format:

    Question: the input question you must answer
    Thought: ...
    Action: ...
    Action Input: ...